<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Baseline" data-toc-modified-id="Baseline-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline</a></span><ul class="toc-item"><li><span><a href="#Train-set" data-toc-modified-id="Train-set-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Train set</a></span></li><li><span><a href="#Val-set" data-toc-modified-id="Val-set-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Val set</a></span></li><li><span><a href="#Random-prediction" data-toc-modified-id="Random-prediction-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Random prediction</a></span></li></ul></li></ul></div>

In [1]:
import os
import pandas as pd
import glob
import numpy as np
import pickle
import gensim
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, f1_score

# Baseline

## Train set


In [2]:
df_Y_train = pd.read_pickle('train/Y_train_filtre.pkl')
df_Y_train = df_Y_train.drop(columns=['Tags'])
df_Y_train = df_Y_train.rename(columns={'tags_filtered': 'tag'})

# build dictionary
dictionary = gensim.corpora.Dictionary(df_Y_train['tag'])

# build bow
Y_train = [dictionary.doc2bow(text) for text in df_Y_train['tag'].values]

# sparse to dense
Y_train = gensim.matutils.corpus2dense(
    Y_train, len(dictionary), len(Y_train)).T

Y_train.shape

(103225, 100)

## Val set 

In [3]:
df_Y_val = pd.read_pickle('val/Y_val_filtre.pkl')
df_Y_val = df_Y_val.drop(columns=['Tags'])
df_Y_val = df_Y_val.rename(columns={'tags_filtered': 'tag'})

# build bow from train set dictionary
Y_val = [dictionary.doc2bow(text) for text in df_Y_val['tag'].values]

# sparse to dense
Y_val = gensim.matutils.corpus2dense(Y_val, len(dictionary), len(Y_val)).T

Y_val.shape

(34477, 100)

## Random prediction

In [4]:
# baseline

size_docs = Y_val.shape[0]
size_tags = Y_val.shape[1]
count_tags = sum(sum(Y_val))
percent_tag = count_tags / (size_docs * size_tags)
Y_pred_val = (np.random.rand(size_docs, size_tags) < percent_tag) * 1

In [5]:
Y_pred_val.shape

(34477, 100)

In [6]:
# evaluate
print('Nombre de tags présents : {}'.format(sum(sum(Y_val))))
print('Nombre de tags prédits : {}'.format(sum(sum(Y_pred_val))))
print('---------------------------------------------------------')

csv_path = 'results/baseline.csv'
clf_name = 'baseline'
val_score_macro = f1_score(Y_val, Y_pred_val, average='macro')
val_score_micro = f1_score(Y_val, Y_pred_val, average='micro')
val_score_weighted = f1_score(Y_val, Y_pred_val, average='weighted')
val_score_samples = f1_score(Y_val, Y_pred_val, average='samples')
with open(csv_path, 'w') as file:
    file.write('{};{};{};{};{};{};{};{};{}'.format(clf_name,
                                                   val_score_macro,
                                                   val_score_macro,
                                                   val_score_micro,
                                                   val_score_micro,
                                                   val_score_weighted,
                                                   val_score_weighted,
                                                   val_score_samples,
                                                   val_score_samples))

print(classification_report(Y_val, Y_pred_val))

Nombre de tags présents : 56448.0
Nombre de tags prédits : 56090
---------------------------------------------------------
              precision    recall  f1-score   support

           0       0.02      0.02      0.02       559
           1       0.01      0.02      0.01       216
           2       0.03      0.01      0.02      1144
           3       0.07      0.02      0.03      2376
           4       0.08      0.02      0.04      1858
           5       0.01      0.01      0.01       364
           6       0.01      0.02      0.01       294
           7       0.01      0.01      0.01       802
           8       0.05      0.02      0.03      1492
           9       0.00      0.01      0.01       191
          10       0.01      0.01      0.01       292
          11       0.00      0.01      0.01       186
          12       0.09      0.02      0.03      3009
          13       0.00      0.00      0.00       438
          14       0.01      0.02      0.01       324
          15

C:\Users\Mini\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
